## RAG: Retrieval Augmented Generation.
- Large language models (LLMs) have a limited context size.
- TLDR
- Not all context is relevant to a given question
- Query -> Search -> Results -> (LLM) -> Answer

## Keyword VS Semantic Search 
![Vector](https://blog.dataiku.com/hs-fs/hubfs/dftt%202.webp?width=1346&height=632&name=dftt%202.webp)

from https://blog.dataiku.com/semantic-search-an-overlooked-nlp-superpower

![Emb_search](figures/emb_search.png)

from https://sreent.medium.com/llms-embeddings-and-vector-search-d4bd9362df56

In [2]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [3]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

In [4]:
import warnings

warnings.filterwarnings("ignore")

# Most powerful solar embedding
![Solar Embedding](figures/solar_emb.jpeg)



In [5]:
from langchain_upstage import UpstageEmbeddings

embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")
embeddings = embeddings_model.embed_documents(
    [
        "What is the best season to visit Korea?",
    ]
)

len(embeddings), len(embeddings[0])

(1, 4096)

In [6]:
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document

sample_text_list = [
    "Korea is a beautiful country to visit in the spring.",
    "The best time to visit Korea is in the fall.",
    "Best way to find bug is using unit test.",
    "Python is a great programming language for beginners.",
    "Sung Kim is a great teacher.",
    "맛있는 좋은 과일을 많이 먹어 볼까?"
]

sample_docs = [Document(page_content=text) for text in sample_text_list]

vectorstore = Chroma.from_documents(
    documents=sample_docs,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

retriever = vectorstore.as_retriever()



In [7]:
result_docs = retriever.invoke("How to find problems in code?")
print(result_docs[0].page_content[:100])


Best way to find bug is using unit test.


In [8]:
result_docs = retriever.invoke("When to visit Korea?")
print(result_docs[0].page_content[:100])

The best time to visit Korea is in the fall.


In [9]:
result_docs = retriever.invoke("Who is a great prof?")
print(result_docs[0].page_content[:100])

Sung Kim is a great teacher.


In [10]:
result_docs = retriever.invoke("좋은 선생님")
print(result_docs[0].page_content[:100])

Sung Kim is a great teacher.


In [11]:
# RAG 1. load doc (done), 2. chunking, splits, 3. embeding - indexing, 4. retrieve

In [12]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader("pdfs/kim-tse-2008.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [13]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 125


In [14]:
from langchain_chroma import Chroma

# 3. Embed & indexing
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

In [ ]:
# 4. retrive
retriever = vectorstore.as_retriever()
result_docs = retriever.invoke("What is Bug Classification?")
print(len(result_docs))
print(result_docs[0].page_content[:100])

4
<p id='48' style='font-size:16px'>Similar in spirit to change classification is work that<br>classif


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [ ]:
chain.invoke({"question": "What is bug classficiation?", "Context": result_docs})

'Bug classification is a process that involves extracting keywords from bug reports or change requests and using them as features to train a machine learning classifier. The goal of this classification is to place a bug report into a specific category or to find the developer best suited to fix a bug. This work, along with change classification, highlights the potential of using machine learning techniques in software engineering.'

# Excercise: Hybrid
Sometimes keyword search can be useful. Design a system that does keyword and semantic search, then combine the results. Use them as context for your RAG.